<a href="https://colab.research.google.com/github/Lusic12/Building-a-Searching-System/blob/main/Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate==0.26.1
!apt install git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH =384
STRIDE =128





In [ ]:
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_training_examples(examples):
    # clear space
    questions = [q.strip() for q in examples["question"]]

    # Tokenizer
    inputs = tokenizer(space
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
Reader
    # use offset mapping
    offset_mapping = inputs.pop("offset_mapping")

    # Trích xuất sample_map từ inputs và loại bỏ nómapping ra khỏi inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")

    # Trích xuất thông tin về câu trả lời (answers) từ examples
    answers = examples["answers"]

    # make list start_position and list end_position
    start_positions = []
    end_positions = []

    # Duyệt qua danh sách offset_mapping
    for i, offset in enumerate(offset_mapping):
        # Xác định index của mẫu (sample) liên quan đến offset hiện tại
        sample_idx = sample_map[i]

        # Trích xuất sequence_ids từ inputs
        sequence_ids = inputs.sequence_ids(i)

        # Xác định vị trí bắt đầu và kết thúc của ngữ cảnh
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Trích xuất thông tin về câu trả lời cho mẫu này
        answer = answers[sample_idx]

        if len(answer['text']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời trong ngữ cảnh
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])

            # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh, gán nhãn là (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Nếu không, gán vị trí bắt đầu và kết thúc dựa trên vị trí của các mã thông tin
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    # Thêm thông tin vị trí bắt đầu và kết thúc vào inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched= True,
    remove_columns = raw_datasets["train"].column_names
)

len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

(130319, 131754)

In [ ]:
def preprocess_validation_examples(examples):
    # Chuẩn bị danh sách câu hỏi bằng cách loại bỏ khoảng trắng ở đầu và cuối mỗi câu hỏi
    questions = [q.strip() for q in examples["question"]]

    # Sử dụng tokenizer để mã hóa các câu hỏi và văn bản liên quan
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Lấy ánh xạ để ánh xạ lại ví dụ tham chiếu cho từng dòng trong inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    # Xác định ví dụ tham chiếu cho mỗi dòng đầu vào và điều chỉnh ánh xạ offset
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        # Loại bỏ các offset không phù hợp với sequence_ids
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None
            for k, o in enumerate(offset)
        ]

    # Thêm thông tin ví dụ tham chiếu vào đầu vào
    inputs["example_id"] = example_ids

    return inputs


In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched= True,
    remove_columns = raw_datasets["validation"].column_names
)

len(raw_datasets["validation"]), len(train_dataset)

(11873, 131754)

In [ ]:
 # Load model
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tạo đối tượng args là các tham số cho quá trình huấn luyện
args = TrainingArguments(
    output_dir="distilbert-finetuned-squadv2",  # Thư mục lưu output
    evaluation_strategy="no",  # Chế độ đánh giá không tự động sau mỗi epoch
    save_strategy="epoch",  # Lưu checkpoint sau mỗi epoch
    learning_rate=2e-5,  # Tốc độ học
    num_train_epochs=3,  # Số epoch huấn luyện
    weight_decay=0.01,  # Giảm trọng lượng mô hình để tránh overfitting
    fp16=True,  # Sử dụng kiểu dữ liệu half-precision để tối ưu tài nguyên
    push_to_hub=True,  # Đẩy kết quả huấn luyện lên HuggingFace Hub
)


In [ ]:
Trainer = Trainer(
    model =  model,
    args =args,
    train_dataset= train_dataset,
    eval_dataset = validation_dataset ,
    tokenizer= tokenizer
)

Trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.074900
1000,2.265600
1500,2.054600
2000,1.898500
2500,1.780400
3000,1.690500
3500,1.623400
4000,1.610300
4500,1.541500
5000,1.518300


TrainOutput(global_step=49410, training_loss=1.043050761436118, metrics={'train_runtime': 5946.2605, 'train_samples_per_second': 66.472, 'train_steps_per_second': 8.309, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.043050761436118, 'epoch': 3.0})

In [ ]:
# Gửi dữ liệu đào tạo lên Hub
Trainer.push_to_hub(commit_message ="Training complete")


CommitInfo(commit_url='https://huggingface.co/Lusic/distilbert-finetuned-squadv2/commit/711a5143538fdf3acf588444e5e1fc297392d4c7', commit_message='Training complete', commit_description='', oid='711a5143538fdf3acf588444e5e1fc297392d4c7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Tải metric " squad " từ thư viện evaluate
metric = evaluate.load("squad_v2")

In [ ]:
N_BEST = 20  # Số lượng kết quả tốt nhất được lựa chọn sau khi dự đoán
MAX_ANS_LENGTH = 30  # Độ dài tối đa cho câu trả lời dự đoán

def compute_metrics(start_logits, end_logits, features, examples):
    # Tạo một từ điển mặc định để ánh xạ mỗi ví dụ với danh sách các đặc trưng tương ứng
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example['id']
        context = example['context']
        answers = []

        # Lặp qua tất cả các đặc trưng liên quan đến ví dụ đó
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]['offset_mapping']

            # Lấy các chỉ số có giá trị lớn nhất cho start và end logits
            start_indexes = np.argsort(start_logit)[-1:-N_BEST-1:-1].tolist()
            end_indexes = np.argsort(end_logit)[-1:-N_BEST-1:-1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Bỏ qua các câu trả lời không hoàn toàn nằm trong ngữ cảnh
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Bỏ qua các câu trả lời có độ dài > MAX_ANS_LENGTH
                    if end_index - start_index + 1 > MAX_ANS_LENGTH:
                        continue

                    # Tạo một câu trả lời mới
                    text = context[offsets[start_index][0]: offsets[end_index][1]]
                    logit_score = start_logit[start_index] + end_logit[end_index]
                    answer = {
                        'text': text,
                        'logit_score': logit_score,
                    }
                    answers.append(answer)

            # Chọn câu trả lời có điểm số tốt nhất
            if len(answers) > 0:
                best_answer = max(answers, key=lambda x: x['logit_score'])
                answer_dict = {
                    'id': example_id,
                    'prediction_text': best_answer['text'],
                    'no_answer_probability': 1 - best_answer['logit_score']
                }
            else:
                answer_dict = {
                    'id': example_id,
                    'prediction_text': '',
                    'no_answer_probability': 1.0
                }
            predicted_answers.append(answer_dict)

    # Tạo danh sách câu trả lời lý thuyết từ các ví dụ
    theoretical_answers = [{'id': ex['id'], 'answers': ex['answers']} for ex in examples]
    # Sử dụng metric.compute để tính toán các độ đo và trả về kết quả
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)


In [ ]:
def perform_validation_predictions(trainer, validation_dataset, raw_validation_dataset):
    # Thực hiện dự đoán trên tập dữ liệu validation
    predictions, _, _ = trainer.predict(validation_dataset)

    # Lấy ra thông tin về các điểm bắt đầu và điểm kết thúc của câu trả lời dự đoán
    start_logits, end_logits = predictions

    # Tính toán các chỉ số đánh giá sử dụng hàm compute_metrics
    results = compute_metrics(start_logits, end_logits, validation_dataset, raw_validation_dataset)

    return results


In [ ]:
# Sử dụng pipeline như một trợ giúp cấp cao
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'lusic/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)


config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]